In [ ]:
import requests
from bs4 import BeautifulSoup
from google.colab import files
import csv
import os
import re
import pandas as pd

In [ ]:
DOMAIN = 'https://www.nfl.com/stats/player-stats/category'

# Lista para armazenar as URLs encontradas
URLs = [f'{DOMAIN}/passing/2024/reg/all/passingyards/desc',
        f'{DOMAIN}/passing/2024/REG/all/passingyards/DESC?aftercursor=AAAAGQAAABlAoMIAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlNVFExSWl3aU16SXdNRFF6TkRFdE5USXpOUzA0TnprM0xUTmlPV0V0TUdReU56RXhZekJpWXpJM0lpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/passing/2024/REG/all/passingyards/DESC?aftercursor=AAAAMgAAADJAdpAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXpOakVpTENJek1qQXdORFEwWmkwME1qTXdMVEl6TmpBdE9HVXhOQzFrWW1KbE1XUmxaR1V4TWpBaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/passing/2024/REG/all/passingyards/DESC?aftercursor=AAAASwAAAEtANgAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlNaUlzSWpNeU1EQTBZVFExTFRRMk1qWXRPVEk0Tnkxak5EVTBMVFEzT1dJNU5ESmpZbVF5TmlJc0lqSXdNalFpWFgwPQ==',
        f'{DOMAIN}/passing/2024/REG/all/passingyards/DESC?aftercursor=AAAAZAAAAFsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUmpOREV0TldFd05DMHpOekUwTFdObVlqY3RPRGN5WWpJd1lqQmpaVEV6SWl3aU1qQXlOQ0pkZlE9PQ=='
]

HEADERS = {'Uer-Agent': 'Mozilla/5.0'}

In [ ]:
def get_page_content(url, headers):
  response = requests.get(url, headers=headers)
  soup = BeautifulSoup(response.text, 'html.parser')
  return soup

In [ ]:
# Lista para armazenar as URLs dos jogadores e das imagens corrigidas
url_players = []

# Loop sobre todas as URLs principais
for url in URLs:
    # Obter o conteúdo da página para a URL atual
    soup = get_page_content(url, HEADERS)

    # Localizar a tabela de jogadores na página
    table = soup.find('table', {'class': 'd3-o-table d3-o-table--detailed d3-o-player-stats--detailed d3-o-table--sortable'})

    # Iterar sobre todas as linhas que contêm os links para os jogadores
    for row in table.find_all('a', {'class': 'd3-o-player-fullname nfl-o-cta--link'}):
        # Adicionar o link de cada jogador à lista
        url_players.append(row['href'])

In [ ]:
# Lista para armazenar os dados dos jogos
games_data = []

# Loop sobre todas as URLs dos jogadores
counter = 0
for url in url_players:
    # Coletar informações do time do jogador em 2024
    soup = get_page_content(f'https://www.nfl.com{url}stats/career', HEADERS)

    # Extrair a posição do jogador
    position_tag = soup.find('span', {'class': 'nfl-c-player-header__position'})
    player_position = position_tag.text.strip() if position_tag else 'Unknown'

    # Extrair o time do jogador
    team_tag = soup.find_all('td')
    player_team = team_tag[1].text.strip() if team_tag else 'Unknown'

    # Dados do time em 2024
    soup = get_page_content(f'https://www.nfl.com{url}stats', HEADERS)

    table = soup.find('table')
    rows = table.find_all('tr')

    counter += 1
    print(f'{counter} - {player_team}')

    if player_position == "QB":
        for row in rows:
            # Extrair as células da linha
            cells = row.find_all('td')
            if len(cells) >= 3:  # Verificar se a linha tem pelo menos 3 colunas (week, opponent, result)
                week = cells[0].text.strip()
                opponent = cells[1].text.strip()
                result = cells[2].text.strip()

                # Adicionar os dados à lista
                games_data.append([player_team, week, opponent, result])

# Criar o arquivo CSV 'Games'
with open('Games.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['team', 'week', 'opponent', 'result'])  # Cabeçalho
    writer.writerows(games_data)  # Adicionar os dados dos jogos


1 - Cincinnati Bengals
2 - Detroit Lions
3 - Tampa Bay Buccaneers
4 - Seattle Seahawks
5 - Minnesota Vikings
6 - Baltimore Ravens
7 - Kansas City Chiefs
8 - New York Jets
9 - Los Angeles Chargers
10 - San Francisco 49ers
11 - Arizona Cardinals
12 - Denver Broncos
13 - Los Angeles Rams
14 - Buffalo Bills
15 - Houston Texans
16 - Washington Commanders
17 - Chicago Bears
18 - Atlanta Falcons
19 - Green Bay Packers
20 - Philadelphia Eagles
21 - Miami Dolphins
22 - Pittsburgh Steelers
23 - Carolina Panthers
24 - New England Patriots
25 - New Orleans Saints
26 - Cleveland Browns
27 - Tennessee Titans
28 - New York Giants
29 - Jacksonville Jaguars
30 - Las Vegas Raiders
31 - Dallas Cowboys
32 - Dallas Cowboys
33 - Indianapolis Colts
34 - Indianapolis Colts
35 - Jacksonville Jaguars
36 - Las Vegas Raiders
37 - Tennessee Titans
38 - New Orleans Saints
39 - Cleveland Browns
40 - Pittsburgh Steelers
41 - New York Giants
42 - Carolina Panthers
43 - Miami Dolphins
44 - New England Patriots
45 - Atl